In [1]:
# Cellule 1 - Préparation du déploiement
print("🚀 PRÉPARATION DU DÉPLOIEMENT DU MODÈLE...")

from azureml.core import Workspace, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from sklearn.ensemble import IsolationForest
import joblib
import pandas as pd
import os

# Connexion au workspace
ws = Workspace.from_config()
print(f"✅ Workspace: {ws.name}")

🚀 PRÉPARATION DU DÉPLOIEMENT DU MODÈLE...


C:\Users\HH\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\HH\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\HH\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


✅ Workspace: stock-anomaly-ml-workspace


In [2]:
# Cellule 2 - Sauvegarde du modèle Isolation Forest
print("💾 SAUVEGARDE DU MODÈLE ISOLATION FOREST...")

# Charger vos résultats pour récupérer le modèle
anomaly_results = pd.read_csv('anomaly_detection_results.csv')

# Créer un modèle Isolation Forest simple pour la démo
# (Dans un vrai projet, vous sauvegarderiez votre vrai modèle entraîné)
model = IsolationForest(
    n_estimators=100,
    contamination=0.05,
    random_state=42
)

# Entraîner sur des données factices pour l'exemple
import numpy as np
X_dummy = np.random.randn(100, 5)
model.fit(X_dummy)

# Sauvegarder le modèle
model_path = "isolation_forest_model.joblib"
joblib.dump(model, model_path)
print(f"✅ Modèle sauvegardé: {model_path}")

💾 SAUVEGARDE DU MODÈLE ISOLATION FOREST...
✅ Modèle sauvegardé: isolation_forest_model.joblib


In [6]:
# Cellule 3 CORRIGÉE - Création du script de scoring
print("📝 CRÉATION DU SCRIPT DE SCORING CORRIGÉ...")

scoring_script = '''
import json
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import IsolationForest

def init():
    global model
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "isolation_forest_model.joblib")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        # Convertir les données JSON en DataFrame
        data_dict = json.loads(raw_data)
        data = data_dict["data"]
        feature_names = data_dict.get("feature_names", ["feature_1", "feature_2", "feature_3", "feature_4", "feature_5"])
        
        df = pd.DataFrame(data, columns=feature_names)
        
        # Prédiction avec Isolation Forest
        predictions = model.predict(df)
        scores = model.decision_function(df)
        
        # Formater les résultats
        results = {
            "predictions": predictions.tolist(),
            "anomaly_scores": scores.tolist(),
            "is_anomaly": [1 if x == -1 else 0 for x in predictions],
            "message": "Détection d\\'anomalies réussie"
        }
        
        return json.dumps(results)
    
    except Exception as e:
        error_result = {"error": str(e)}
        return json.dumps(error_result)
'''

# Sauvegarder le script corrigé
with open('score.py', 'w') as f:
    f.write(scoring_script)
    
# Vérification du fichier
with open('score.py', 'r') as f:
    content = f.read()
    lines = content.split('\n')
    print(f"✅ Script créé: {len(lines)} lignes")
    print("📄 Aperçu des premières lignes:")
    for i, line in enumerate(lines[:10], 1):
        print(f"{i:2d}: {line}")

print("✅ Script de scoring corrigé et sauvegardé!")

📝 CRÉATION DU SCRIPT DE SCORING CORRIGÉ...
✅ Script créé: 40 lignes
📄 Aperçu des premières lignes:
 1: 
 2: import json
 3: import pandas as pd
 4: import numpy as np
 5: import joblib
 6: import os
 7: from sklearn.ensemble import IsolationForest
 8: 
 9: def init():
10:     global model
✅ Script de scoring corrigé et sauvegardé!


In [7]:
# Cellule 4 - Configuration de l'environnement
print("🔧 CONFIGURATION DE L'ENVIRONNEMENT...")

# Créer un environnement conda
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('stock-anomaly-env')
cd = CondaDependencies.create(
    pip_packages=[
        'azureml-defaults',
        'scikit-learn==1.0.2',
        'pandas==1.3.5',
        'numpy==1.21.4',
        'joblib==1.1.0'
    ]
)

env.python.conda_dependencies = cd
print("✅ Environnement configuré")

🔧 CONFIGURATION DE L'ENVIRONNEMENT...
✅ Environnement configuré


In [8]:
# Cellule 5 - Configuration du déploiement
print("🎯 CONFIGURATION DU DÉPLOIEMENT ACI...")

# Configuration d'inférence
inference_config = InferenceConfig(
    entry_script='score.py',
    environment=env
)

# Configuration ACI (Azure Container Instance)
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Service de détection d\'anomalies sur données financières',
    tags={'model': 'IsolationForest', 'domain': 'finance'}
)

print("✅ Configuration ACI prête")

🎯 CONFIGURATION DU DÉPLOIEMENT ACI...
✅ Configuration ACI prête


In [11]:
# Cellule 6 CORRIGÉE - Déploiement avec gestion des conflits
print("🚀 DÉPLOIEMENT DU MODÈLE EN TEMPS RÉEL...")

try:
    # Vérifier si le service existe déjà
    from azureml.core.webservice import Webservice
    
    try:
        existing_service = Webservice(ws, 'stock-anomaly-service')
        print(f"⚠️ Service existant trouvé: {existing_service.name}")
        print(f"📊 État: {existing_service.state}")
        
        # Supprimer l'ancien service s'il est en échec ou en cours
        if existing_service.state in ['Failed', 'Unhealthy', 'Transitioning']:
            print("🗑️ Suppression de l'ancien service...")
            existing_service.delete()
            print("✅ Ancien service supprimé")
        else:
            print("✅ Service déjà déployé et sain - réutilisation")
            service = existing_service
            
    except Exception as e:
        print("🔧 Création d'un nouveau service...")
        
        # Enregistrer le modèle si pas déjà fait
        try:
            model = Model(ws, 'stock_anomaly_detection_model')
            print(f"✅ Modèle existant récupéré: {model.name}")
        except:
            model = Model.register(
                workspace=ws,
                model_path='isolation_forest_model.joblib',
                model_name='stock_anomaly_detection_model',
                description='Modèle Isolation Forest pour détection d\'anomalies'
            )
            print(f"✅ Nouveau modèle enregistré: {model.name}")
        
        # Attendre 30 secondes pour éviter les conflits
        import time
        print("⏳ Attente de 30 secondes pour éviter les conflits...")
        time.sleep(30)
        
        # Déployer le service
        service = Model.deploy(
            workspace=ws,
            name='stock-anomaly-service',
            models=[model],
            inference_config=inference_config,
            deployment_config=deployment_config,
            overwrite=True
        )
        
        # Attendre le déploiement
        service.wait_for_deployment(show_output=True)
        print(f"🎉 SERVICE DÉPLOYÉ: {service.scoring_uri}")

except Exception as e:
    print(f"❌ Erreur de déploiement: {e}")
    print("\n💡 VÉRIFIEZ MANUELLEMENT DANS AZURE ML STUDIO:")
    print("1. Allez sur ml.azure.com")
    print("2. Endpoints → Vérifiez l'état de 'stock-anomaly-service'")
    print("3. Si 'Failed', supprimez-le et réessayez")

🚀 DÉPLOIEMENT DU MODÈLE EN TEMPS RÉEL...
⚠️ Service existant trouvé: stock-anomaly-service
📊 État: Transitioning
🗑️ Suppression de l'ancien service...
Running
2025-11-18 18:15:40+01:00 Check and wait for operation (c30fbcf3-5b9d-4222-8711-38fcd9ae4b39) to finish.
2025-11-18 18:15:41+01:00 Deleting service entity.
Succeeded
✅ Ancien service supprimé


In [14]:
# Test après permissions
print("🧪 TEST APRÈS AJOUT DES PERMISSIONS...")

from azureml.core import Workspace, Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
import time

ws = Workspace.from_config()

# Attendre que les permissions soient propagées
print("⏳ Attente de 2 minutes pour propagation des permissions...")
time.sleep(120)

try:
    # Essayer un déploiement simple
    model = Model(ws, 'stock_anomaly_detection_model')
    
    service = Model.deploy(
        workspace=ws,
        name='stock-anomaly-service-test',
        models=[model],
        inference_config=inference_config,
        deployment_config=deployment_config,
        overwrite=True
    )
    
    service.wait_for_deployment(show_output=True)
    print(f"🎉 RÉUSSI! Service: {service.scoring_uri}")
    
except Exception as e:
    print(f"❌ Encore une erreur: {e}")

🧪 TEST APRÈS AJOUT DES PERMISSIONS...
⏳ Attente de 2 minutes pour propagation des permissions...


C:\Users\HH\AppData\Local\Temp\ipykernel_15948\503293529.py:19: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-11-18 18:41:30+01:00 Creating Container Registry if not exists.
2025-11-18 18:41:32+01:00 Use the existing image.
2025-11-18 18:41:32+01:00 Generating deployment configuration.
2025-11-18 18:41:35+01:00 Submitting deployment to compute.
Failed


Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 52a3b32d-bfd6-41a7-bf75-325395d79cdc
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client '43ebf4b9-f883-4da7-abe1-3c74329e5989' with object id 'b2c5b721-0b90-4bd3-84e6-29d271fe73ed' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/06a1cf6d-be75-42c9-8148-ccdde9e7d1f7/resourceGroups/stock-anomaly-rg/providers/Microsoft.ContainerInstance/containerGroups/stock-anomaly-service-test-rTD6Ls_q-UOrCMGvJLiTpQ' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}



❌ Encore une erreur: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 52a3b32d-bfd6-41a7-bf75-325395d79cdc
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client '43ebf4b9-f883-4da7-abe1-3c74329e5989' with object id 'b2c5b721-0b90-4bd3-84e6-29d271fe73ed' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/06a1cf6d-be75-42c9-8148-ccdde9e7d1f7/resourceGroups/stock-anomaly-rg/providers/Microsoft.ContainerInstance/containerGroups/stock-anomaly-service-test-rTD6Ls_q-UOrCMGvJLiTpQ' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal s